<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/%EA%B8%B0%EB%B3%B8%EB%A7%8C%20%EC%93%B0%EB%A9%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install insightface onnxruntime

  Using cached numpy-2.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.0-cp311-cp311-manylinux_2_28_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.


In [6]:
import requests
import os
import cv2
import insightface
from insightface.app import FaceAnalysis
from glob import glob
import subprocess

# --- 모델 다운로드 함수 (변동 없음) ---
def download_inswapper():
    """인스왑퍼 모델을 다운로드합니다."""
    url = "https://civitai.com/api/download/models/85159"
    model_name = 'inswapper_128.onnx'

    if os.path.exists(model_name):
        print(f"'{model_name}' 모델이 이미 존재합니다. 다운로드를 건너뜀니다.")
        return True

    print(f"'{model_name}' 모델 다운로드 중...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(model_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("모델 다운로드 완료!")
        return True
    else:
        print(f"다운로드 실패: {response.status_code}")
        return False

# --- 얼굴 교체 후 이미지 파일로 저장하는 함수 (자동 FPS 계산 추가) ---
def perform_face_swapping_and_save_images(source_img_path, target_video_path, output_dir='output', frame_interval=5):
    if not download_inswapper():
        return False, None, None

    print("얼굴 분석 및 교체 모델 준비 중...")
    app = FaceAnalysis()
    app.prepare(ctx_id=0, det_size=(320, 320))
    swapper = insightface.model_zoo.get_model('inswapper_128.onnx')

    # 소스 이미지 처리
    source_img = cv2.imread(source_img_path)
    if source_img is None:
        print(f"소스 이미지를 찾을 수 없습니다: {source_img_path}")
        return False, None, None

    source_faces = app.get(source_img)
    if not source_faces:
        print("소스 이미지에서 얼굴을 찾을 수 없습니다.")
        return False, None, None
    source_face = source_faces[0]
    print("소스 얼굴 준비 완료")

    # 타겟 비디오 처리
    cap = cv2.VideoCapture(target_video_path)
    if not cap.isOpened():
        print(f"타겟 비디오를 열 수 없습니다: {target_video_path}")
        return False, None, None

    # 원본 비디오 정보 가져오기
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    new_fps = original_fps / frame_interval

    print(f"원본 FPS: {original_fps:.1f} → 새 FPS: {new_fps:.1f}")
    print(f"총 프레임: {total_frames}, 예상 저장 이미지: {total_frames // frame_interval}개")

    os.makedirs(output_dir, exist_ok=True)

    frame_count = 0
    saved_image_count = 0

    print("비디오 프레임 처리 및 얼굴 교체 중...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            target_faces = app.get(frame)
            if len(target_faces) > 0:
                result_frame = swapper.get(frame, target_faces[0], source_face, paste_back=True)
                cv2.imwrite(os.path.join(output_dir, f'frame_{saved_image_count:04d}.png'), result_frame)
                saved_image_count += 1

                # 진행상황 표시 (10개마다)
                if saved_image_count % 10 == 0:
                    progress_percent = (frame_count / total_frames) * 100
                    print(f"진행: {saved_image_count}개 이미지 저장됨... ({progress_percent:.1f}%)")

        frame_count += 1

    cap.release()
    print(f"저장된 이미지: {saved_image_count}개")

    return True, original_fps, new_fps

# --- 이미지 파일들을 사용하여 최종 비디오 생성 함수 (FPS 자동 적용) ---
def create_video_from_swapped_images(image_directory='output', output_video_name='final_swapped_video.mp4', target_fps=None):
    print(f"이미지들로 영상 생성 중...")
    image_files = sorted(glob(os.path.join(image_directory, 'frame_*.png')))

    if len(image_files) == 0:
        print(f"이미지 파일이 없습니다: {image_directory}")
        return None

    first_img = cv2.imread(image_files[0])
    if first_img is None:
        return None

    height, width, _ = first_img.shape
    fps = target_fps if target_fps else 30.0

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_name, fourcc, fps, (width, height))

    if not out.isOpened():
        return None

    print(f"{len(image_files)}개의 이미지를 {fps:.1f}FPS로 처리 중...")
    for i, img_path in enumerate(image_files):
        img = cv2.imread(img_path)
        if img is not None:
            out.write(img)

            # 진행상황 표시 (50개마다)
            if (i + 1) % 50 == 0:
                progress_percent = ((i + 1) / len(image_files)) * 100
                print(f"영상 생성 진행: {i + 1}/{len(image_files)} ({progress_percent:.1f}%)")

    out.release()
    print(f"영상 생성 완료: {output_video_name}")
    return output_video_name

# --- RIFE를 위한 ffmpeg 보간 함수 추가 ---
def interpolate_with_ffmpeg(input_video, output_video, original_fps, frame_interval):
    """
    FFmpeg minterpolate 필터로 보간 수행 (원본 FPS에 맞게 자동 계산)
    """
    base_fps = original_fps / frame_interval
    target_fps = original_fps  # 최종 FPS는 원본과 동일
    multiplier = int(target_fps / base_fps)

    print(f"보간 시작: {input_video} → {target_fps:.2f}fps (x{multiplier})")

    cmd = [
        'ffmpeg', '-i', input_video,
        '-filter:v', f'minterpolate=fps={target_fps}:mi_mode=mci:mc_mode=aobmc:vsbmc=1',
        '-y', output_video
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"보간 완료: {output_video}")
        return True
    else:
        print(f"보간 실패: {result.stderr}")
        return False

# --- 메인 실행 부분 ---
if __name__ == "__main__":
    FRAME_INTERVAL = 5
    USE_INTERPOLATION = True
    INTERPOLATION_MULTIPLIER = 4

    print("Face Swap 시작!")
    print(f"샘플링 간격: {FRAME_INTERVAL}프레임마다")

    # 1. 얼굴 교체 및 이미지 저장
    success, original_fps, calculated_fps = perform_face_swapping_and_save_images(
        source_img_path='/content/source.png',
        target_video_path='/content/target.mp4',
        output_dir='output',
        frame_interval=FRAME_INTERVAL
    )

    if success and calculated_fps:
        # 2. 계산된 FPS로 비디오 생성
        base_video = create_video_from_swapped_images(
            image_directory='output',
            output_video_name='base_swapped_video.mp4',
            target_fps=calculated_fps
        )

        if base_video and os.path.exists(base_video):
            print(f"기본 영상 생성 완료: {base_video}")

            # 3. 프레임 보간
            if USE_INTERPOLATION:
                interpolated_video = 'final_interpolated_video.mp4'
                if interpolate_with_ffmpeg(
                  base_video,
                  interpolated_video,
                  original_fps=original_fps,
                  frame_interval=FRAME_INTERVAL
                ):
                    print(f"최종 결과: {interpolated_video}")
    print(f"최종 결과: {base_video}")

Face Swap 시작!
샘플링 간격: 5프레임마다
'inswapper_128.onnx' 모델이 이미 존재합니다. 다운로드를 건너뜀니다.
얼굴 분석 및 교체 모델 준비 중...
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (320, 320)
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
inswapper-shape: [1, 3, 128, 128]
소스 얼굴 준비 완료
원본 FPS: 30.0 → 새 FPS: 6.0
총 프레임: 357, 예상 저장 이미지: 71개
비디오 프레임 처리 및 얼굴 교체 중...
진행: 10개 이미지 저장됨... (12.6%)
진행: 20개 이미지 저장됨... (26.6%)
진행: 30개 이미지 저장됨... (40.6%)
진행: 40개 이미지 저장됨... (54.6%)
진행: 50개 이미지 저장됨... (68.6%)
진행: 60개 이미지 저장됨... (82.6%)
진행: 70개 이미지 저장됨... (96.6%)
저장된 이미지: 72개
이미지들로 영상 생성 중...
94개의 이미지를 6.0FPS로 처리 중...
영상 생성 진행: 50/94 (53.2%)
영상 생성 완료: base_swapped_video.mp4
기본 영상 생성 완료: base_swapped_video.mp4
보간 시작: base_swapped_video.mp4 → 30.00fps (x5)
보간 완료: final_interpolated_video.mp4
최종 결과: final_interpolated_video.mp4
최종 결과: base_swapped_video.mp4
